In [2]:
import pandas as pd
import sqlite3
from datetime import datetime
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from preprocessing import nlputils


[nltk_data] Downloading package punkt to /home/malte/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/malte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()
    
## Latent Dirichlet Allocation
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20
batch_size = 128


In [18]:
    
conn = sqlite3.connect("news.db")
c = conn.cursor()

# get news headlines
columns = ['headline', 'teaser']
query = f"""
    SELECT {", ".join(columns)}
    FROM ArticleContent
    LIMIT 20
"""
c.execute(query)
result = c.fetchall()

df = pd.DataFrame(result, columns=columns)
# datetime_pattern = "%Y-%m-%d %H:%M:%S"
# datetime.strptime("2022-11-05 07:22:00", datetime_pattern)
# df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, datetime_pattern))
# with open("data/news/headline.csv", 'w') as f:
#     f.writelines([line + "\n" for line in df_headlines['headline'] if line])

data_samples = df['teaser'].to_list()
# [sample.split(':') for sample in data_samples] #split headline by : 

germanTextProcessing = nlputils.GermanTextProcessing()
data_processed = germanTextProcessing.batch_run(data_samples)
# tfidf_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1))
# tfidf = tfidf_vectorizer.fit_transform(data_processed)
# lda = LatentDirichletAllocation(
#     n_components=10,
# )
# lda.fit(tfidf)

# tf_feature_names = tfidf_vectorizer.get_feature_names_out()
# plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")
# data_processed

100%|██████████| 20/20 [00:00<00:00, 40.17it/s]


In [19]:
pd.options.display.max_colwidth = 500

In [20]:
df_compare = pd.DataFrame({"original": data_samples, "processed": data_processed})

In [21]:
df_compare

,original,processed
0,"Wie in jedem Quartal hält die jüngste Bilanzsaison für Börsianer mit Spannung erwartete Zahlenwerke bereit. Besonders im Fokus stehen die US-Tech-Giganten wie Apple, NVIDIA, Microsoft, PayPal, Facebook, Amazon, Google, Tesla und Co.",wie Quartal halten jung Bilanzsaison Börsianer Spannung erwartet Zahlenwerke bereit besonders Fokus stehen US Tech Gigant Apple NVIDIA Microsoft PayPal Facebook Amazon Google Tesla Co
1,Der Versandhändler Amazon hat sein Netz in Niedersachsen mit einem neuen Logistikzentrum in Helmstedt ausgebaut.,der Versandhändler Amazon Netz Niedersachsen Logistikzentrum Helmstedt ausbauen
2,Im vergangenen Monat haben Experten die xx-Aktie unter die Lupe genommen. Die Ergebnisse.,in Monat Experte xx Aktie Lupe nehmen der Ergebnis
3,Der weltgrößte Online-Versandhändler Amazon will seine Mitarbeiterzahl angesichts von Inflations- und Rezessionsrisiken vorerst nicht weiter erhöhen.,der weltgrößter Online Versandhändler Amazon Mitarbeiterzahl angesichts Inflation Rezessionsrisiken vorerst erhöhen
4,"Amazon unternimmt einen neuen Anlauf, im Musikstreaming-Markt den Rückstand zu den Marktführern Spotify und Apple Music aufzuholen.",Amazon unternehmen Anlauf Musikstreaming Markt Rückstand Marktführern Spotify Apple Music aufholen
5,"Der Kauf von Aktien ist heutzutage mit Hilfe von Direktbrokern und Investment-Apps so leicht wie nie zuvor. Viele Anleger vergessen in diesem Zusammenhang jedoch, dass nicht nur der Kauf, sondern vor allem auch der richtige Verkauf entscheidend für den langfristigen Erfolg am Kapitalmarkt ist.",der Kauf Aktie heutzutage Hilfe Direktbroker Investment Apps zuvor vieler Anleger vergessen Zusammenhang Kauf richtig Verkauf entscheidend langfristig Erfolg Kapitalmarkt
6,Der Internetkonzern Amazon hat über seine Geschäftsentwicklung im dritten Quartal 2022 berichtet.,der Internetkonzer Amazon Geschäftsentwicklung Quartal 2022 berichten
7,Amazon öffnet die Bücher - was Experten erwarten.,Amazon öffnen Buch Experte erwarten
8,"Jack Ma, der Chef des chinesischen Online-Riesen Alibaba, hätte in seinem Leben viele Male aufgeben können. Doch trotz unzähligem Scheitern verlor Ma sein Ziel nicht aus den Augen. Eine Erfolgsgeschichte.",Jack Ma Chef chinesisch Online Riese Alibaba Leben Mal aufgeben doch trotz unzählig scheitern verlieren Ma Ziel Auge ein Erfolgsgeschichte
9,"Der Onlinehändler Amazon engagiert Hawaiian Airlines, um seine Pakete mit zehn gemieteten Airbus-Maschinen durch die USA zu fliegen.",der Onlinehändler Amazon engagieren Hawaiian Airlines Pakete gemietet Airbus Maschine USA fliegen
